<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Run your first Python UDF</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook.</p>
    </div>
</div>

<p>This feature is currently in Private Preview. Please reach out support@singlestore.com to confirm if this feature can be enabled in your org.</p>

This Jupyter notebook will help you build your first Python UDF using Notebooks, registering it with your database and calling it as part of SQL query.

## Create some simple tables

This setup establishes a basic relational structure to store some reviews for restaurants. Ensure you have selected a database.

In [1]:
%%sql
DROP TABLE IF EXISTS reviews;

CREATE TABLE IF NOT EXISTS
reviews (
    review_id INT PRIMARY KEY,
    store_name VARCHAR(255) NOT NULL,
    review TEXT NOT NULL
);

## Insert sample data

In [2]:
%%sql INSERT into reviews (review_id, store_name, review) values
("1", "Single Pizza", "The staff were very respectful and made thoughtful suggestions. I will definitely go again. 10/10!"),
("2", "Single Pizza", "The food was absolutely amazing and the service was fantastic!"),
("3", "Single Pizza", "The experience was terrible. The food was cold and the waiter was rude."),
("4", "Single Pizza", "I loved the ambiance and the desserts were out of this world!"),
("5", "Single Pizza", "Not worth the price. I expected more based on the reviews");

## Define Python UDF functions

Next, we will be Python UDF function using the `@udf` annotation. We will be using the `VADER` model of `nltk` library to perform sentiment analysis on the review text.

In [3]:
!pip install nltk

In [4]:
from singlestoredb.functions import udf
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

@udf
def review_sentiment(review: str) -> str:
    print("review:" + review)
    scores = sia.polarity_scores(review)
    sentiment = (
        "Positive" if scores['compound'] > 0.05 else
        "Negative" if scores['compound'] < -0.05 else
        "Neutral"
    )
    print("sentiment:" + sentiment)
    return sentiment

## Start the Python UDF server

This will start the server as well as register all the functions annotated with `@udf` as external user defined functions on your selected database.

In [5]:
import singlestoredb.apps as apps
connection_info = await apps.run_udf_app(replace_existing=True)

## List all registered UDFs

In interactive notebooks, the udf function will be suffixed with `_test` to differentiate it from the published version

In [6]:
%%sql
SHOW functions

## Call the UDF from SQL

You will now be able to run queries like

```
SELECT review_id, store_name, review, review_sentiment_test(review) from reviews order by review_id;
```
from the SQL editor or any other SQL client.

Try it out by opening another notebook, selecting the current Database and running this query in a new cell.

## Publish Python UDF

After validating the Python UDF interactively, you can publish it and access it like

```
%%sql
SELECT review_id, store_name, review, review_sentiment(review) from reviews order by review_id
```

enriching your data exploration experience seamlessly!

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>